<a href="https://colab.research.google.com/github/filipediniz/Coursera_Capstone/blob/master/Toronto_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests_html
!pip install pixiedust
!pip install geocoder
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

     |████████████████████████████████| 1.2MB 8.4MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=485e5433585ed8da16a3aa1cc9f409f25e9fce67b9245f54aebe9ac2caeaf59f
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for parse: filename=parse-1.15.0-cp36-none-any.whl size=23710 sha256=0c03cb7acc1db614db20c1734abad04766dba48a1c74eaf43769c355f05972fd
  Stored in directory: /root/.cache/pip/wheels/07/ee/c8/eced0759f09fc635398ab1b8e89c38549b28e5db7fd4a53ba5
  Created wheel for pyppeteer: filename=pyppeteer-0.0.25-cp36-none-any.whl size=78360 sha256=0786197a5ed727f4656d6b902b029a664a604df06fd8c77ea7570a67ba6bf481
  Stored in directory: /root/.cache/pip/wheels/34/e0/5d/070e22eceecf7ecd5fa4b86bbc18c1c7d0b90e02e9b57f35eb
Successfully built fake-useragent parse pyppeteer
     |████████████████████████████████| 204

In [0]:
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import numpy as np
import matplotlib.pyplot as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
import pixiedust
import requests
from requests_html import HTMLSession
import re as regex

import datetime as dt
from pandas_datareader import data

from IPython.display import clear_output

import pickle

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

1. Start by creating a new Notebook for this assignment.

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

In [0]:
#%%pixie_debugger

session = HTMLSession()
DF = pd.DataFrame()
ret = session.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')    
try:
    DF = pd.read_html(ret.text, decimal=",", thousands='.', encoding='utf-8')[0]
    #DF.columns = DF.iloc[0]
    #DF = DF.reset_index()
except:
    print('Página sem dados')
    

In [4]:
DF

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


 3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [0]:
# Get names of indexes for which column Age has value 30
indexDrop = DF[DF['Borough'] == 'Not assigned'].index
 
# Delete these row indexes from dataFrame
DF.drop(indexDrop , inplace=True)

In [6]:
DF

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
DF[DF['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


No row has a neighbourhood that is 'Not assigned''

- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
DF.shape

(210, 3)

4. Submit a link to your Notebook on your Github repository. (10 marks)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

In [0]:
#import geocoder # import geocoder
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#
#for r in range(0, DF.shape[0]):
#  print('Current row: {}'.format(r))
#  # initialize your variable to None
#  lat_lng_coords = None
#
#  # loop until you get the coordinates
#  g = None
#  while(type(g) == type(None)):    
#    print('Attempting to retrieve coordinates for address {}...'.format('{}, Toronto, Canada'.format(DF.iloc[r]['Postcode'])))
#    geolocator = Nominatim(user_agent="toronto_explorer")
#    g = geolocator.geocode('{}, Toronto, Canada'.format(DF.iloc[r]['Postcode']))      
#    if (type(g) != type(None)):    
#      print('Latitude = {}; longitude = {}'.format(g.latitude, g.longitude))  
#      DF.iloc[r]['Latitude'] = g.latitude
#      DF.iloc[r]['Longitude'] = g.longitude

In [0]:
LL = pd.read_csv('http://cocl.us/Geospatial_data')

In [11]:
for i in range(0, DF.shape[0]):
  for j in range(0, LL.shape[0]):
    if DF.iloc[i]['Postcode'] == LL.iloc[j]['Postal Code']:
      print('Postal code {} coordinates are: latitude = {}; longitude = {}'.format(LL.iloc[j]['Postal Code'], LL.iloc[j]['Latitude'], LL.iloc[j]['Longitude']))
      DF.ix[i, 'Latitude'] = LL.iloc[j]['Latitude']
      DF.ix[i, 'Longitude'] = LL.iloc[j]['Longitude']
      break

Postal code M3A coordinates are: latitude = 43.7532586; longitude = -79.3296565
Postal code M4A coordinates are: latitude = 43.725882299999995; longitude = -79.31557159999998
Postal code M5A coordinates are: latitude = 43.6542599; longitude = -79.3606359
Postal code M6A coordinates are: latitude = 43.718517999999996; longitude = -79.46476329999999
Postal code M6A coordinates are: latitude = 43.718517999999996; longitude = -79.46476329999999
Postal code M7A coordinates are: latitude = 43.6623015; longitude = -79.3894938
Postal code M9A coordinates are: latitude = 43.6678556; longitude = -79.53224240000002
Postal code M1B coordinates are: latitude = 43.806686299999996; longitude = -79.19435340000001
Postal code M1B coordinates are: latitude = 43.806686299999996; longitude = -79.19435340000001
Postal code M3B coordinates are: latitude = 43.745905799999996; longitude = -79.352188
Postal code M4B coordinates are: latitude = 43.7063972; longitude = -79.309937
Postal code M4B coordinates are:

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  


Postal code M5B coordinates are: latitude = 43.6571618; longitude = -79.37893709999999
Postal code M5B coordinates are: latitude = 43.6571618; longitude = -79.37893709999999
Postal code M6B coordinates are: latitude = 43.709577; longitude = -79.44507259999999
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M1C coordinates are: latitude = 43.7845351; longitude = -79.16049709999999
Postal code M1C coordinates are: latitude = 43.7845351; longitude = -79.16049709999999
Postal code M1C coordinates are: latitude = 43.7845351; longitude = -79.16049709999999
Postal code M3C coordinates are: latitude = 

In [12]:
DF.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.654260,-79.360636
3,M4A,North York,Victoria Village,43.718518,-79.464763
4,M5A,Downtown Toronto,Harbourfront,43.718518,-79.464763
5,M6A,North York,Lawrence Heights,43.662301,-79.389494
6,M6A,North York,Lawrence Manor,43.667856,-79.532242


Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

In [0]:
DF.dropna(inplace=True)
DF_toronto = DF[DF['Borough'].str.contains('Toronto')]

In [14]:
DF_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,Harbourfront,43.718518,-79.464763
7,M7A,Downtown Toronto,Queen's Park,43.806686,-79.194353
16,M5B,Downtown Toronto,Ryerson,43.650943,-79.554724
17,M5B,Downtown Toronto,Garden District,43.650943,-79.554724
33,M5C,Downtown Toronto,St. James Town,43.763573,-79.188711
46,M4E,East Toronto,The Beaches,43.754328,-79.442259
47,M5E,Downtown Toronto,Berczy Park,43.705369,-79.349372
56,M5G,Downtown Toronto,Central Bay Street,43.778517,-79.346556
57,M6G,Downtown Toronto,Christie,43.767980,-79.487262
67,M5H,Downtown Toronto,Adelaide,43.727929,-79.262029


In [15]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(DF_toronto['Latitude'], DF_toronto['Longitude'], DF_toronto['Borough'], DF_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [24]:
CLIENT_ID = '4FJMQVQY0O2GDFFQLW3NBT20XONVI1SDAX2ODVF1M2XYPJEI' # your Foursquare ID
CLIENT_SECRET = 'UTVH1JPXDP3SC4OZUUQHHY2R5D0NWQS5EHMDWQTTB3KMWZ44' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4FJMQVQY0O2GDFFQLW3NBT20XONVI1SDAX2ODVF1M2XYPJEI
CLIENT_SECRET:UTVH1JPXDP3SC4OZUUQHHY2R5D0NWQS5EHMDWQTTB3KMWZ44


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
toronto_venues = getNearbyVenues(names=DF_toronto['Neighbourhood'],
                                   latitudes=DF_toronto['Latitude'],
                                   longitudes=DF_toronto['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market


In [34]:
#toronto_venues.set_index(list(toronto_venues)[0], inplace=True)
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.718518,-79.464763,Roots,43.718221,-79.466776,Boutique
1,Harbourfront,43.718518,-79.464763,Kitchen Stuff Plus (Clearance Outlet),43.719096,-79.462675,Furniture / Home Store
2,Harbourfront,43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant
3,Harbourfront,43.718518,-79.464763,Orfus Road Shopping Outlets,43.719045,-79.460849,Clothing Store
4,Harbourfront,43.718518,-79.464763,Tim Hortons,43.719427,-79.467995,Coffee Shop


In [56]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
toronto_onehot = pd.concat([toronto_onehot['Neighborhood'].to_frame(), toronto_onehot.drop(columns=['Neighborhood'], axis=1)], axis=1)
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,...,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
toronto_onehot.shape

(879, 192)

In [58]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,...,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.052632,0.000000,0.00,0.000000,0.000000,0.000000,...,0.052632,0.000000,0.00,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.052632,0.00000,0.000000,0.000000,0.000000,0.052632
2,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.043478,...,0.086957,0.000000,0.00,0.000000,0.086957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.043478,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.031746,0.015873,0.015873,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.00,0.015873,0.015873,0.00,0.015873,0.000000,0.000000,...,0.015873,0.000000,0.00,0.015873,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.015873,0.000000,0.00000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.015873,0.015873,0.015873,0.000000,0.00000,0.015873,0.00000,0.000000,0.015873,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.031746,0.000000
4,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [60]:
toronto_grouped.shape

(53, 192)

In [61]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                venue  freq
0         Coffee Shop   0.2
1   Convenience Store   0.2
2  Chinese Restaurant   0.2
3      Discount Store   0.2
4    Department Store   0.2


----Berczy Park----
               venue  freq
0  Indian Restaurant  0.11
1        Yoga Studio  0.05
2               Bank  0.05
3        Gas Station  0.05
4       Liquor Store  0.05


----Brockton----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09
2          Restaurant  0.09
3         Coffee Shop  0.09
4                 Pub  0.04


----Central Bay Street----
                  venue  freq
0        Clothing Store  0.16
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.06
3     Electronics Store  0.03
4                Bakery  0.03


----Chinatown----
               venue  freq
0     Baseball Field   0.5
1       Home Service   0.5
2  Accessories Store   0.0
3             Museum   0.0
4     Medical Center   0.0


----Christie----
                    venue  freq
0 

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [122]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Chinese Restaurant,Discount Store,Convenience Store,Department Store,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center
1,Berczy Park,Indian Restaurant,Yoga Studio,Fast Food Restaurant,Pharmacy,Pizza Place,Gas Station,Restaurant,Burger Joint,Sandwich Place,Liquor Store
2,Brockton,Italian Restaurant,Restaurant,Sandwich Place,Coffee Shop,Thai Restaurant,Pharmacy,Comfort Food Restaurant,Café,Pub,Butcher
3,Central Bay Street,Clothing Store,Coffee Shop,Fast Food Restaurant,Electronics Store,Japanese Restaurant,Women's Store,Juice Bar,Bakery,Food Court,Convenience Store
4,Chinatown,Baseball Field,Home Service,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store


In [123]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 0, 0, 2, 2, 0], dtype=int32)

In [124]:
DF_toronto.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [125]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = DF_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,Harbourfront,43.718518,-79.464763,0.0,Furniture / Home Store,Accessories Store,Arts & Crafts Store,Women's Store,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Clothing Store,Vietnamese Restaurant
7,M7A,Downtown Toronto,Queen's Park,43.806686,-79.194353,0.0,Print Shop,Fast Food Restaurant,Yoga Studio,Dance Studio,Drugstore,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
16,M5B,Downtown Toronto,Ryerson,43.650943,-79.554724,3.0,Home Service,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
17,M5B,Downtown Toronto,Garden District,43.650943,-79.554724,3.0,Home Service,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
33,M5C,Downtown Toronto,St. James Town,43.763573,-79.188711,0.0,Electronics Store,Breakfast Spot,Medical Center,Intersection,Rental Car Location,Mexican Restaurant,Spa,Yoga Studio,Department Store,Dog Run


In [126]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

toronto_merged.dropna(inplace=True)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[(cluster)-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters